In [11]:
##%overwritefile
##%file:src/c_do_execute.py
##%noruncode
    def do_execute(self, code, silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        try:
            self.silent = silent
            ##扫描代码
            # magics, code = self._filter_magics(code)
            magics, code = self.mag.filter(code)
            if len(self.addkey2dict(magics,'replcmdmode'))>0:
                return self.send_replcmd(code, silent, store_history=True,
                    user_expressions=None, allow_stdin=False)
            
            ############# run gdb and send command begin
            if len(magics['rungdb'])>0:
                return self.replgdb_sendcmd(code,silent, store_history,
                    user_expressions, allow_stdin)
            ############# run gdb and send command
            #############send replcmd's command
            if magics['runmode']=='repl':
                if hasattr(self, 'replcmdwrapper'):
                    if self.replcmdwrapper :
                        return self.repl_sendcmd(code, silent, store_history,
                            user_expressions, allow_stdin,magics)
            #FIXME:
            #############send replcmd's command end
            ############# only run command mark
            
            if len(self.addkey2dict(magics,'onlyruncmd'))>0:
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [],'user_expressions': {}}
            ############# only create source file
            
            if len(self.addkey2dict(magics,'onlycsfile'))<1 :
                magics, code = self._add_main(magics, code)
            return_code=0
            fil_ename=''

            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,1)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}

            with self.new_temp_file(suffix='.c') as source_file:
                source_file.write(code)
                source_file.flush()
                newsrcfilename=source_file.name 
                fil_ename=newsrcfilename
                return_code=True
                # Generate new src file
                bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,2)
                if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}

                # if len(self.addkey2dict(magics,'onlycsfile'))>0:
                #     if len(self.addkey2dict(magics,'file'))<1:
                #         self._log("no file name parameter\n",2)
                #     return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
                
                # Generate executable file :being
                bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,2,1)
                if bcancel_exec:
                    return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
                if len(self.addkey2dict(magics,'file'))>0:
                    fil_ename=magics['file'][0]
                else: fil_ename=source_file.name
                returncode,binary_filename=self._exec_gcc_(fil_ename,magics)
                fil_ename=binary_filename
                return_code=returncode
                bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,2,2)
                if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
                
                if returncode!=0:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [],'user_expressions': {}}
                # Generate executable file :end
            ############# only run gcc，no not run executable file
            if len(self.addkey2dict(magics,'onlyrungcc'))>0:
                self._log("only run gcc \n")
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [],'user_expressions': {}}
            
            
            ##代码运行前
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,1)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            self._write_to_stdout("The process :"+binary_filename+"\n")
            ################# repl mode run code files
            #FIXME:
            if magics['runmode']=='repl':
                self._start_replprg(binary_filename,magics['args'],magics)

                return_code=p.returncode
                bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
                # if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
                
                return  {'status': 'ok', 'execution_count': self.execution_count,
                            'payload': [], 'user_expressions': {}}
            ############################################

        ############################################
            #################dynamically load and execute code
            #FIXME:
            if len(magics['dlrun'])>0:
                p = self.create_jupyter_subprocess([self.master_path, binary_filename] + magics['args'],env=self.addkey2dict(magics,'env'))
            #################
            else:
                p = self.create_jupyter_subprocess([binary_filename] + magics['args'],env=self.addkey2dict(magics,'env'))
            self.subprocess=p
            self.g_rtsps[str(p.pid)]=p
            return_code=p.returncode
            ##代码启动后
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            
            if len(self.addkey2dict(magics,'showpid'))>0:
                self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
            while p.poll() is None:
                p.write_contents(magics)
            
            ############################################
            p.write_contents(magics)
            # wait for threads to finish, so output is always shown
            p._stdout_thread.join()
            p._stderr_thread.join()
            p.write_contents(magics)

            return_code=p.returncode
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,3)
            if bcancel_exec:return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            ##代码运行结束
            # now remove the files we have just created
            if(os.path.exists(source_file.name)):
                os.remove(source_file.name)
            # if(os.path.exists(binary_filename)):
                # os.remove(binary_filename)

            # if p.returncode != 0:
                # self._write_to_stderr("[C kernel] Executable exited with code {}".format(p.returncode))
        except Exception as e:
            self._log(""+str(e),3)
        return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/src/c_do_execute.py created successfully


In [12]:
##%overwritefile
##%file:../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py
###%file:c_kernel.py
##%noruncode
#
#   MyC Jupyter Kernel 
#   generated by MyPython
#
##%include:src/head.py

##%include:src/common.py

##%include:src/IREPLWrapper.py
##%include:src/RealTimeSubprocess.py

class CKernel(Kernel):
    implementation = 'jupyter-MyC-kernel'
    implementation_version = '1.0'
    language = 'C'
    language_version = 'C11'
    language_info = {'name': 'text/x-csrc',
                     'mimetype': 'text/x-csrc',
                     'file_extension': '.c'}
    banner = "C kernel.\n" \
             "Uses gcc, compiles in C11, and creates source code files and executables in temporary folder.\n"

    main_head = "#include <stdio.h>\n" \
            "#include <math.h>\n" \
            "int main(int argc, char* argv[], char** env){\n"

    main_foot = "\nreturn 0;\n}"

##%include:src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(CKernel, self).__init__(*args, **kwargs)
        self.kernelinfo='[MyC Kernel]'
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        self.files = []
        mastertemp = tempfile.mkstemp(suffix='.out')
        os.close(mastertemp[0])
        self.master_path = mastertemp[1]
        self.resDir = path.join(path.dirname(path.realpath(__file__)), 'resources')
        filepath = path.join(self.resDir, 'master.c')
        subprocess.call(['gcc', filepath, '-std=c11', '-rdynamic', '-ldl', '-o', self.master_path])

        self.chk_replexit_thread = Thread(target=self.chk_replexit, args=(self.g_rtsps))
        self.chk_replexit_thread.daemon = True
        self.chk_replexit_thread.start()
        self.init_plugin()
        self.mag=Magics(self,self.plugins)
    
##%include:src/_templateHander.py
##%include:src/_readtemplatefile.py

##%include:src/cleannotes.py
##%include:src/kernel_common.py
    ##%include:src/_is_specialID.py

##%include:src/repl_listpid.py
##%include:src/chk_replexit.py
##%include:src/cleanup_files.py
##%include:src/new_temp_file.py
##%include:src/log_output.py
##%include:src/_write_display_data.py
##%include:src/_write_to_stdout.py
##%include:src/_write_to_stderr.py
##%include:src/_read_from_stdin.py
##%include:src/readcodefile.py
    #####################################################################
##%include:src/_start_replprg.py
##%include:src/process_output.py
##%include:src/send_replcmd.py
    #####################################################################
##%include:src/do_shell_command.py

##%include:src/create_jupyter_subprocess.py

##%include:src/compile_with_gcc.py
##%include:src/_filter_env.py
#####################################################################
###%include:src/c_filter_magics.py
#####################################################################

##%include:src/_add_main.py

##%include:src/c_exec_gcc_.py
##%include:src/c_start_gdb.py
##%include:src/c_do_replexecutegdb.py
##%include:src/c_replgdb_sendcmd.py
##%include:src/_icallfunc.py
##%include:src/c_do_execute.py
##%include:src/do_shutdown.py
#####################################################################
##%include:src/pluginmng.py


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py created successfully
